In [1]:
import pandas as pd 
import ast
import re
import numpy as np
import nltk
import json
from datetime import datetime
from datetime import timedelta
from textblob import TextBlob
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.util import ngrams
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import spacy
import pyLDAvis.gensim_models
# import en_core_web_md
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

2022-10-22 00:17:05.343395: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/wenyin/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/wenyin/opt/anaconda3/lib/python3.8/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
/Users/wenyin/opt/anaconda3/lib/python3.8/site-packages/matplotlib_inline/config.py:59: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 

True

In [2]:
#df = pd.read_csv("/Users/ivankoh/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/1D/NUS Y3S1/BT4222/Data/train/kickstarter_train.csv")
df = pd.read_csv("../kickstarter_train.csv")

In [82]:
# extract main category and subcategory from category variable
# df['subcategory'] = df['category'].apply(lambda x: json.loads(x)['slug'].split("/")[1] if len(json.loads(x)['slug'].split("/")) > 1 else None)
#df['category'] = df['category'].apply(lambda x: json.loads(x)['slug'].split("/")[0])
# df['location'] = df['location'].apply(lambda x: json.loads(x)['country'])

## NLP features

In [3]:
# Text cleaning for: rewards, description, description story, description risks

def clean_text(df):
    def process_rewards(corpus):
        corpus_processed = []
        for row in corpus:
            row_processed = ""
            row = row.replace("\\n", " ")
            row = ast.literal_eval(row)

            for dict in row:
                row_processed += dict['rewards'].lower() + ' '

            
            row_processed = row_processed.replace("//",'')
            row_processed = re.sub(r'[^\w\s]', '', row_processed) # remove punctuation
            corpus_processed.append(row_processed)

        return corpus_processed
    
    def process_description_story(corpus):
        corpus_processed = []
        for row in corpus:
            row = str(row)
            row_processed = row.replace("\r", " " )
            row_processed = row_processed.replace("\n", " " )
            row_processed = re.sub(r'[^\w\s]', '', row_processed) # remove punctuation
            corpus_processed.append(row_processed if not pd.isnull(row_processed) else "") # handle NA

        return corpus_processed

    df["rewards_processed"] = process_rewards(df["rewards"])
    df["description_processed"] = process_description_story(df["description"])
    df["description_story_processed"] = process_description_story(df["description_story"])
    df["description_risks_processed"] = process_description_story(df["description_risks"])

    return df


In [4]:
class LemmatizeTokenizer(object):
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
    def __call__(self, text):
        return [self.lemmatizer.lemmatize(word) for word in word_tokenize(text)]

In [5]:
# need to return the vectorizer to use on the test data to transform it 
def generate_nlp_features(df):
    
    # Rewards

    vect_rewards = TfidfVectorizer( 
        tokenizer=LemmatizeTokenizer(),
        lowercase=True,
        analyzer='word', 
        ngram_range=(1,3), # unigram, bigram and trigram 
        max_features=100, # vocabulary that only consider the top max_features ordered by term frequency across the corpus
        min_df=10, # minimum word frequency required to be in model
        stop_words=stopwords.words('english') # remove stopwords
        )

    rewards_processed = pd.Series(df["rewards_processed"])
    tfidf_fit_rewards = vect_rewards.fit(rewards_processed)
    rewards_tfidf_array = tfidf_fit_rewards.transform(rewards_processed).toarray()
    rewards_tfidf_df = pd.DataFrame(rewards_tfidf_array)
    rewards_tfidf_df.columns = list(map(lambda x : "rewards_" + str(x), rewards_tfidf_df.columns))
    df = pd.merge(df, rewards_tfidf_df , left_index=True, right_index=True)
    

    # Description

    vect_description = TfidfVectorizer( 
        tokenizer=LemmatizeTokenizer(),
        lowercase=True,
        analyzer='word', 
        ngram_range=(1,3), # unigram, bigram and trigram 
        max_features=100, # vocabulary that only consider the top max_features ordered by term frequency across the corpus
        min_df=10, # minimum word frequency required to be in model
        stop_words=stopwords.words('english') # remove stopwords
        )

    description = pd.Series(df["description_processed"])
    tfidf_fit_description = vect_description.fit(description)
    description_tfidf_array = tfidf_fit_description.transform(description).toarray()
    description_tfidf_df = pd.DataFrame(description_tfidf_array)
    description_tfidf_df.columns = list(map(lambda x : "description_" + str(x), description_tfidf_df.columns))
    df = pd.merge(df, description_tfidf_df , left_index=True, right_index=True)


    # Description Story

    vect_description_story = TfidfVectorizer( 
        tokenizer=LemmatizeTokenizer(),
        lowercase=True,
        analyzer='word', 
        ngram_range=(1,3), # unigram, bigram and trigram 
        max_features=100, # vocabulary that only consider the top max_features ordered by term frequency across the corpus
        min_df=10, # minimum word frequency required to be in model
        stop_words=stopwords.words('english') # remove stopwords
        )
    
    description_story_processed = pd.Series(df["description_story_processed"])
    tfidf_fit_description_story_processed = vect_description_story.fit(description_story_processed)
    description_story_processed_tfidf_array = tfidf_fit_description_story_processed.transform(description_story_processed).toarray()
    description_story_tfidf_df = pd.DataFrame(description_story_processed_tfidf_array)
    description_story_tfidf_df.columns = list(map(lambda x : "description_story_" + str(x), description_story_tfidf_df.columns))
    df = pd.merge(df, description_story_tfidf_df , left_index=True, right_index=True)


    # Description Risks

    vect_description_risks = TfidfVectorizer( 
        tokenizer=LemmatizeTokenizer(),
        lowercase=True,
        analyzer='word', 
        ngram_range=(1,3), # unigram, bigram and trigram 
        max_features=100, # vocabulary that only consider the top max_features ordered by term frequency across the corpus
        min_df=10, # minimum word frequency required to be in model
        stop_words=stopwords.words('english') # remove stopwords
        )

    description_risks_processed = pd.Series(df["description_risks_processed"])
    tfidf_fit_description_risks_processed = vect_description_risks.fit(description_risks_processed)
    description_risks_processed_tfidf_array = tfidf_fit_description_risks_processed.transform(description_risks_processed).toarray()
    description_risks_tfidf_df = pd.DataFrame(description_risks_processed_tfidf_array)
    description_risks_tfidf_df.columns = list(map(lambda x : "description_risks_" + str(x), description_risks_tfidf_df.columns))
    df = pd.merge(df, description_risks_tfidf_df , left_index=True, right_index=True)

    return (df, tfidf_fit_rewards, tfidf_fit_description, tfidf_fit_description_story_processed, tfidf_fit_description_risks_processed)


def nlp_transform_test(df, tfidf_fit_rewards, tfidf_fit_description, tfidf_fit_description_story_processed, tfidf_fit_description_risks_processed):
    rewards_tfidf_array = tfidf_fit_rewards.transform(df['rewards_processed']).toarray()
    rewards_tfidf_df = pd.DataFrame(rewards_tfidf_array)
    rewards_tfidf_df.columns = list(map(lambda x : "rewards_" + str(x), rewards_tfidf_df.columns))
    df = pd.merge(df, rewards_tfidf_df , left_index=True, right_index=True)
    
    description_tfidf_array = tfidf_fit_description.transform(df['description_processed']).toarray()
    description_tfidf_df = pd.DataFrame(description_tfidf_array)
    description_tfidf_df.columns = list(map(lambda x : "description_" + str(x), description_tfidf_df.columns))
    df = pd.merge(df, description_tfidf_df , left_index=True, right_index=True)
    
    description_story_processed_tfidf_array = tfidf_fit_description_story_processed.transform(df['description_story_processed']).toarray()
    description_story_tfidf_df = pd.DataFrame(description_story_processed_tfidf_array)
    description_story_tfidf_df.columns = list(map(lambda x : "description_story_" + str(x), description_story_tfidf_df.columns))
    df = pd.merge(df, description_story_tfidf_df , left_index=True, right_index=True)
    
    description_risks_processed_tfidf_array = tfidf_fit_description_risks_processed.transform(df['description_risks_processed']).toarray()
    description_risks_tfidf_df = pd.DataFrame(description_risks_processed_tfidf_array)
    description_risks_tfidf_df.columns = list(map(lambda x : "description_risks_" + str(x), description_risks_tfidf_df.columns))
    df = pd.merge(df, description_risks_tfidf_df , left_index=True, right_index=True)
    
    return df


In [6]:
for x in df.columns:
    print(x)

id
name
description
description_story
description_risks
rewards
category
goal
deadline
location
state
staff_pick
video
launched_at
created_at


## Rewards Features

In [7]:
# Remove rewards that got cut off while scraping and closes the dictionary
def create_rewards_tiers(df):
    # df["rewards"] = df["rewards"].apply(lambda x : clean_rewards(x))
    df["reward_tiers"] = df["rewards"].apply(lambda x : len(ast.literal_eval(x)))
    df = move_reward_tiers(df)
    return df

def create_min_max_reward(df):
    df["min_reward"] = 0
    df["max_reward"] = 0

    for i in range(len(df)):
        print(i)
        all_reward_amount = []
        reward = df.iloc[i, 5]
        dict_list = ast.literal_eval(reward)
        for dictionary in dict_list:
            values_string = str(dictionary.values())
            reward_title_SGD = re.search(r"(S)?\$\s?\d{1,3}(,\d{1,3})?", values_string) #S$
            if reward_title_SGD:
                reward_amount = re.search(r"\d{1,3}(,\d{1,3})?", reward_title_SGD.group())
            else:
                 print("Error: No reward in SGD found!")   

            if reward_amount is not None:
                integer_amount = re.sub("[^\d\.]", "", reward_amount.group()) # remove comma in large amounts of money
                all_reward_amount.append(int(integer_amount))
            else:
                pledged_without_reward = re.search(r"Pledge without a reward", values_string) # check if have "Pledge without a reward"
                if pledged_without_reward:
                    all_reward_amount.append(0)
        if len(all_reward_amount) > 0:
            df["min_reward"][i] = min(all_reward_amount)
            df["max_reward"][i] = max(all_reward_amount)
    print("create min max reward done, now moving columns")
    df = move_min_max_reward(df)
    return df

# Rearange reward_tiers column to the right of rewards
def move_reward_tiers(df):
    cols = df.columns.tolist()
    cols = cols[:6] + [cols[-1]] + cols[6:-1]
    df = df[cols]
    return df

# Rearange min and max reward columns to the right of reward_tiers
def move_min_max_reward(df):
    cols = df.columns.tolist()
    cols = cols[:7] + cols[-2:] + cols[7:-2]
    df = df[cols]
    return df

<>:26: DeprecationWarning: invalid escape sequence \d
<>:26: DeprecationWarning: invalid escape sequence \d
/var/folders/11/bwt0l9496qj97nxff6qb8ndw0000gn/T/ipykernel_2112/842725897.py:26: DeprecationWarning: invalid escape sequence \d
  integer_amount = re.sub("[^\d\.]", "", reward_amount.group()) # remove comma in large amounts of money


## Sentiment Features

In [8]:
'''
Polarity is float which lies in the range of [-1,1] where 1 means positive statement and -1 means a negative statement. Subjective sentences generally refer to personal opinion, emotion or judgment whereas objective refers to factual information. Subjectivity is also a float which lies in the range of [0,1].
'''
def generate_sentiment_features(df):
    df = df.dropna(subset=['description_story_processed', 'description_risks_processed', 'description_processed', 'rewards_processed']) # NOTE: put at top with other dropnas from other features?
    df["description_story_polarity"] = df["description_story_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.polarity)
    df["description_story_subjectivity"] = df["description_story_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.subjectivity)
    df["description_polarity"] = df["description_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.polarity)
    df["description_subjectivity"] = df["description_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.subjectivity)
    df["description_risks_polarity"] = df["description_risks_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.polarity)
    df["description_risks_subjectivity"] = df["description_risks_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.subjectivity)
    df["rewards_polarity"] = df["rewards_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.polarity)
    df["rewards_subjectivity"] = df["rewards_processed"].apply(lambda x: 
                   TextBlob(x).sentiment.subjectivity)          
    return df

In [9]:
def generate_word_count_features(df): # omitted description due to word limit, word count likely similar for all projects
    df['description_story_word_count'] = df["description_story_processed"].apply(lambda x: len(str(x).split(" ")))
    df['description_risks_word_count'] = df["description_risks_processed"].apply(lambda x: len(str(x).split(" ")))
    df['rewards_word_count'] = df["rewards_processed"].apply(lambda x: len(str(x).split(" ")))
    return df

## One-hot Encoding of Categorical Variables

when to drop one of the OHE columns: 
https://stats.stackexchange.com/questions/231285/dropping-one-of-the-columns-when-using-one-hot-encoding

In [10]:
def ohe_transform(enc, col, df):
    transformed = enc.transform(df[col].to_numpy().reshape(-1, 1))
    #Create a Pandas DataFrame of the hot encoded column
    ohe_df = pd.DataFrame(transformed, columns=enc.get_feature_names_out())
    #concat with original data
    data = pd.concat([df, ohe_df], axis=1)#.drop([col], axis=1)
    return data

def ohe_fit(col, df):
    enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
    enc.fit(df[col].unique().reshape(-1, 1))
    return (ohe_transform(enc, col, df), enc)

hold out on topic modelling first because it is unsupervised algorithm

<h2> Generate Duration Feature </h2>

In [11]:
def generate_duration_features(df):
    df['deadline'] = df['deadline'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    df['launched_at'] = df['launched_at'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    df['created_at'] = df['created_at'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    df['funding_duration_in_days'] = (df['deadline'] - df['launched_at'].dt.tz_localize(None))
    df['funding_duration_in_days'] = df['funding_duration_in_days'].apply(lambda x: x / timedelta(days=1))
    df['preparation_duration_in_days'] = (df['launched_at'] - df['created_at'].dt.tz_localize(None))
    df['preparation_duration_in_days'] = df['preparation_duration_in_days'].apply(lambda x: x / timedelta(days=1))
    
    return df

## Text Competitor Feature

In [12]:
'''
text competitors: inverse originality of text corpus. 

If score is higher means how the project owner 
words the project is very similar to all the other projects out there, meaning more competitive
because the wordings of the project is very 'unoriginal'
'''

def generate_text_competitor(df):
    # combine all text into 1 corpus 
    df['corpus'] = df['description_processed'] + df['description_story_processed'] + df['description_risks_processed']
    # generate a sparse matrix of tfidf 
    vect_corpus = TfidfVectorizer( 
            tokenizer=LemmatizeTokenizer(),
            lowercase=True,
            analyzer='word', 
            stop_words=stopwords.words('english') # remove stopwords
            )
    tfidf_sparse = vect_corpus.fit_transform(df['corpus'])
    # calculate cosine similarity based on the tfidf 
    cos_sim_dense = cosine_similarity(tfidf_sparse)
    # remove diagonal - A project is exactly the same as itself 
    cos_sim_dense_i_removed = cos_sim_dense[~np.eye(cos_sim_dense.shape[0],dtype=bool)].reshape(cos_sim_dense.shape[0],-1)
    # taking median as the similarity threshold - if greater than equal to median, means similar to that specific project 
    threshold = np.median(cos_sim_dense_i_removed.flatten())
    # if cosine similarity is above threshold return 1, then sum to get similar counts 
    df['text_similarity_score'] = np.apply_along_axis(np.sum, 1, cos_sim_dense_i_removed >= threshold)

    # sns.histplot(result['text_similarity_score'])

    return (df, vect_corpus, threshold)

def generate_text_competitor_test(vectorizer, df, threshold):
    # combine all text into 1 corpus 
    df['corpus'] = df['description_processed'] + df['description_story_processed'] + df['description_risks_processed']
    tfidf_sparse = vectorizer.transform(df['corpus'])
    # calculate cosine similarity based on the tfidf 
    cos_sim_dense = cosine_similarity(tfidf_sparse)
    # remove diagonal - A project is exactly the same as itself 
    cos_sim_dense_i_removed = cos_sim_dense[~np.eye(cos_sim_dense.shape[0],dtype=bool)].reshape(cos_sim_dense.shape[0],-1)
    # taking median as the similarity threshold - if greater than equal to median, means similar to that specific project 
    # threshold from training
    # if cosine similarity is above threshold return 1, then sum to get similar counts 
    df['text_similarity_score'] = np.apply_along_axis(np.sum, 1, cos_sim_dense_i_removed >= threshold)

    # sns.histplot(result['text_similarity_score'])

    return df

## Category and goal competitor feature

In [13]:
'''
Category and goal competitor: projects in the same category, that launched in the same month and in the same goal bin
'''
def generate_cat_goal_competitor(df):
    #df['launched_at'] = df['launched_at'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    df['launched_month']= df['launched_at'].dt.month
    df['launched_year']= df['launched_at'].dt.year
    df['cat_goal_bin'] = df.groupby(['category'])['goal'].transform(
                        lambda x: pd.qcut(x, [0, .33, .67, 1]))
    
    competitors = df.groupby(['category','launched_year','launched_month','cat_goal_bin']).count()
    competitors['competitiors_count'] = competitors['name']
    competitors = competitors[['competitiors_count']]
    competitors.reset_index(inplace=True)
    df = pd.merge(df, competitors, on = ['category', 'launched_year', 'launched_month','cat_goal_bin'], how = 'left')

    return df

## Similar success rate feature

In [14]:
'''
Success rate for similar projects belonging to a particular category, with similar goal and launched in the same year.
'''
def generate_similiar_success_rate(df):
    df_success = df[df['state'] == 1]
    df_success = df_success.groupby(['category','launched_year','cat_goal_bin']).count()
    df_success['success_count'] = df_success['name']
    df_success = df_success[['success_count']]
    df_success.reset_index(inplace=True)
    df = pd.merge(df, df_success, on = ['category', 'launched_year', 'cat_goal_bin'], how = 'left')

    df_total = df.groupby(['category','launched_year','cat_goal_bin']).count()
    df_total['total_count'] = df_total['name']
    df_total = df_total[['total_count']]
    df_total.reset_index(inplace=True)
    df = pd.merge(df, df_total, on = ['category', 'launched_year', 'cat_goal_bin'], how = 'left')
    df['success_rate'] = df['success_count'] / df['total_count']

    return df

## Combine all feature generating functions

In [15]:
result = create_rewards_tiers(df)
print("Rewards tiers done")
result = create_min_max_reward(result)
print("mins max rewards done")

result = clean_text(result)
print("clean text done")
result['has_video'] = result['video'].apply(lambda x: 0 if pd.isnull(x) else 1)

result, tfidf_fit_rewards, tfidf_fit_description, tfidf_fit_description_story_processed, tfidf_fit_description_risks_processed = generate_nlp_features(result)
print("generate_nlp_features done")

result = generate_sentiment_features(result)
print("generate_sentiment_features done")
result = generate_word_count_features(result)
print("generate_word_count_features done")
result = generate_duration_features(result)
print("generate_duration_features done")
result, vect_corpus, threshold = generate_text_competitor(result)
print("generate_text_competitor done")
result = generate_cat_goal_competitor(result)
print("generate_cat_goal_competitor done")
result = generate_similiar_success_rate(result)
print("generate_similiar_success_rate done")

result, category_encoder = ohe_fit('category', result) #use encoder to fit train data
print("ohe_fit category done")
result, location_encoder = ohe_fit('location', result) #use encoder to fit train data
print("ohe_fit location done")


21482
21483
21484
21485
21486
21487
21488
21489
21490
21491
21492
21493
21494
21495
21496
21497
21498
21499
21500
21501
21502
21503
21504
21505
21506
21507
21508
21509
21510
21511
21512
21513
21514
21515
21516
21517
21518
21519
21520
21521
21522
21523
21524
21525
21526
21527
21528
21529
21530
21531
21532
21533
21534
21535
21536
21537
21538
21539
21540
21541
21542
21543
21544
21545
21546
21547
21548
21549
21550
21551
21552
21553
21554
21555
21556
21557
21558
21559
21560
21561
21562
21563
21564
21565
21566
21567
21568
21569
21570
21571
21572
21573
21574
21575
21576
21577
21578
21579
21580
21581
21582
21583
21584
21585
21586
21587
21588
21589
21590
21591
21592
21593
21594
21595
21596
21597
21598
21599
21600
21601
21602
21603
21604
21605
21606
21607
21608
21609
21610
21611
21612
21613
21614
21615
21616
21617
21618
21619
21620
21621
21622
21623
21624
21625
21626
21627
21628
21629
21630
21631
21632
21633
21634
21635
21636
21637
21638
21639
21640
21641
21642
21643
21644
21645
21646
21647
216

## Drop Unnecessary Features, Feature Scaling

In [26]:
to_drop = ['rewards', 'deadline', 'launched_at', 'rewards_processed', 'created_at',
           'description_processed', 'description_story_processed','description_risks_processed',
           'id', 'name', 'description', 'description_story', 'description_risks', 'video',
          'location', 'category', 'corpus', 'launched_month', 'launched_year', 'cat_goal_bin', 'success_count', 'total_count']

final_result = result.drop(to_drop, axis = 1)

#need feature scaling
scaler = MinMaxScaler() 
need_scaling = ['reward_tiers', 'min_reward', 'max_reward', 'goal', 'text_similarity_score',
               'description_story_polarity', 'description_story_subjectivity', 'description_polarity',
               'description_subjectivity', 'description_risks_polarity', 'description_risks_subjectivity',
               'rewards_polarity', 'rewards_subjectivity', 'description_story_word_count',
               'description_risks_word_count', 'rewards_word_count', 'funding_duration_in_days',
               'preparation_duration_in_days', 'competitiors_count', 'success_rate']
final_result[need_scaling] = scaler.fit_transform(result[need_scaling])

## Apply same feature engineering on Test Data

In [17]:
test = pd.read_csv("kickstarter_test.csv")
test = create_rewards_tiers(test)
test = create_min_max_reward(test)

test = clean_text(test)
test['has_video'] = test['video'].apply(lambda x: 0 if pd.isnull(x) else 1)

test = nlp_transform_test(test, tfidf_fit_rewards, tfidf_fit_description, tfidf_fit_description_story_processed, tfidf_fit_description_risks_processed)

test = generate_sentiment_features(test)
test = generate_word_count_features(test)
test = generate_duration_features(test)
test = generate_text_competitor_test(vect_corpus, test, threshold)
test = generate_cat_goal_competitor(test)
test = generate_similiar_success_rate(test)

#encoders
test = ohe_transform(category_encoder, 'category', test)
test = ohe_transform(location_encoder, 'location', test)

#drop and scale 
final_test = test.drop(to_drop, axis=1)
final_test[need_scaling] = scaler.transform(test[need_scaling])

223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472


In [18]:
print(len(final_test.columns))
print(len(final_result.columns))

for i in result.columns:
    if i not in test.columns:
        print(i)

538
538


In [19]:
final_result.columns

Index(['reward_tiers', 'min_reward', 'max_reward', 'category', 'goal',
       'location', 'state', 'staff_pick', 'has_video', 'rewards_0',
       ...
       'x0_VE', 'x0_VN', 'x0_VU', 'x0_WS', 'x0_XK', 'x0_YE', 'x0_ZA', 'x0_ZW',
       'x0_nan', 'success_count'],
      dtype='object', length=538)

## Export

In [98]:
import time 

final_result.to_csv(f'/Users/ivankoh/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/1D/NUS Y3S1/BT4222/Data/train/kickstarter_train_final_{time.strftime("%Y%m%d-%H%M%S")}.csv', index=False)
final_test.to_csv(f'/Users/ivankoh/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/1D/NUS Y3S1/BT4222/Data/test/kickstarter_test_final_{time.strftime("%Y%m%d-%H%M%S")}.csv', index=False)


In [99]:
#since there could be more than one categories for each project, create new features to split the categories.
# df['new_category'] = df.category.str.split("/", expand=False)
# split_cat = pd.DataFrame(df['new_category'].tolist(), columns=['category1', 'category2'])
# #each project should at least have 1 category, 'category2' can be "None". 
# #'category1' being the main category for the project.
# df = pd.concat([df, split_cat], axis=1)
# df.head()

In [21]:

pd.set_option('display.max_columns', None)

In [32]:
final_test.reward_tiers.value_counts()

16.0    4369
Name: reward_tiers, dtype: int64

In [22]:
final_test.head()

,reward_tiers,min_reward,max_reward,category,goal,location,state,staff_pick,has_video,rewards_0,rewards_1,rewards_2,rewards_3,rewards_4,rewards_5,rewards_6,rewards_7,rewards_8,rewards_9,rewards_10,rewards_11,rewards_12,rewards_13,rewards_14,rewards_15,rewards_16,rewards_17,rewards_18,rewards_19,rewards_20,rewards_21,rewards_22,rewards_23,rewards_24,rewards_25,rewards_26,rewards_27,rewards_28,rewards_29,rewards_30,rewards_31,rewards_32,rewards_33,rewards_34,rewards_35,rewards_36,rewards_37,rewards_38,rewards_39,rewards_40,rewards_41,rewards_42,rewards_43,rewards_44,rewards_45,rewards_46,rewards_47,rewards_48,rewards_49,rewards_50,rewards_51,rewards_52,rewards_53,rewards_54,rewards_55,description_0,description_1,description_2,description_3,description_4,description_5,description_6,description_7,description_8,description_9,description_10,description_11,description_12,description_13,description_14,description_15,description_16,description_17,description_18,description_19,description_20,description_21,description_22,description_23,description_24,description_25,description_26,description_27,description_28,description_29,description_30,description_31,description_32,description_33,description_34,description_35,description_36,description_37,description_38,description_39,description_40,description_41,description_42,description_43,description_44,description_45,description_46,description_47,description_48,description_49,description_50,description_51,description_52,description_53,description_54,description_55,description_56,description_57,description_58,description_59,description_60,description_61,description_62,description_63,description_64,description_65,description_66,description_67,description_68,description_69,description_70,description_71,description_72,description_73,description_74,description_75,description_76,description_77,description_78,description_79,description_80,description_81,description_82,description_83,description_84,description_85,description_86,description_87,description_88,description_89,description_90,description_91,description_92,description_93,description_94,description_95,description_96,description_97,description_98,description_99,description_story_0,description_story_1,description_story_2,description_story_3,description_story_4,description_story_5,description_story_6,description_story_7,description_story_8,description_story_9,description_story_10,description_story_11,description_story_12,description_story_13,description_story_14,description_story_15,description_story_16,description_story_17,description_story_18,description_story_19,description_story_20,description_story_21,description_story_22,description_story_23,description_story_24,description_story_25,description_story_26,description_story_27,description_story_28,description_story_29,description_story_30,description_story_31,description_story_32,description_story_33,description_story_34,description_story_35,description_story_36,description_story_37,description_story_38,description_story_39,description_story_40,description_story_41,description_story_42,description_story_43,description_story_44,description_story_45,description_story_46,description_story_47,description_story_48,description_story_49,description_story_50,description_story_51,description_story_52,description_story_53,description_story_54,description_story_55,description_story_56,description_story_57,description_story_58,description_story_59,description_story_60,description_story_61,description_story_62,description_story_63,description_story_64,description_story_65,description_story_66,description_story_67,description_story_68,description_story_69,description_story_70,description_story_71,description_story_72,description_story_73,description_story_74,description_story_75,description_story_76,description_story_77,description_story_78,description_story_79,description_story_80,description_story_81,description_story_82,description_story_83,description_story_84,description_story_85,description_story_86,d

In [27]:
final_result.head()

,reward_tiers,min_reward,max_reward,goal,state,staff_pick,has_video,rewards_0,rewards_1,rewards_2,rewards_3,rewards_4,rewards_5,rewards_6,rewards_7,rewards_8,rewards_9,rewards_10,rewards_11,rewards_12,rewards_13,rewards_14,rewards_15,rewards_16,rewards_17,rewards_18,rewards_19,rewards_20,rewards_21,rewards_22,rewards_23,rewards_24,rewards_25,rewards_26,rewards_27,rewards_28,rewards_29,rewards_30,rewards_31,rewards_32,rewards_33,rewards_34,rewards_35,rewards_36,rewards_37,rewards_38,rewards_39,rewards_40,rewards_41,rewards_42,rewards_43,rewards_44,rewards_45,rewards_46,rewards_47,rewards_48,rewards_49,rewards_50,rewards_51,rewards_52,rewards_53,rewards_54,rewards_55,description_0,description_1,description_2,description_3,description_4,description_5,description_6,description_7,description_8,description_9,description_10,description_11,description_12,description_13,description_14,description_15,description_16,description_17,description_18,description_19,description_20,description_21,description_22,description_23,description_24,description_25,description_26,description_27,description_28,description_29,description_30,description_31,description_32,description_33,description_34,description_35,description_36,description_37,description_38,description_39,description_40,description_41,description_42,description_43,description_44,description_45,description_46,description_47,description_48,description_49,description_50,description_51,description_52,description_53,description_54,description_55,description_56,description_57,description_58,description_59,description_60,description_61,description_62,description_63,description_64,description_65,description_66,description_67,description_68,description_69,description_70,description_71,description_72,description_73,description_74,description_75,description_76,description_77,description_78,description_79,description_80,description_81,description_82,description_83,description_84,description_85,description_86,description_87,description_88,description_89,description_90,description_91,description_92,description_93,description_94,description_95,description_96,description_97,description_98,description_99,description_story_0,description_story_1,description_story_2,description_story_3,description_story_4,description_story_5,description_story_6,description_story_7,description_story_8,description_story_9,description_story_10,description_story_11,description_story_12,description_story_13,description_story_14,description_story_15,description_story_16,description_story_17,description_story_18,description_story_19,description_story_20,description_story_21,description_story_22,description_story_23,description_story_24,description_story_25,description_story_26,description_story_27,description_story_28,description_story_29,description_story_30,description_story_31,description_story_32,description_story_33,description_story_34,description_story_35,description_story_36,description_story_37,description_story_38,description_story_39,description_story_40,description_story_41,description_story_42,description_story_43,description_story_44,description_story_45,description_story_46,description_story_47,description_story_48,description_story_49,description_story_50,description_story_51,description_story_52,description_story_53,description_story_54,description_story_55,description_story_56,description_story_57,description_story_58,description_story_59,description_story_60,description_story_61,description_story_62,description_story_63,description_story_64,description_story_65,description_story_66,description_story_67,description_story_68,description_story_69,description_story_70,description_story_71,description_story_72,description_story_73,description_story_74,description_story_75,description_story_76,description_story_77,description_story_78,description_story_79,description_story_80,description_story_81,description_story_82,description_story_83,description_story_84,description_story_85,description_story_86,description_story_8

In [28]:
result.head()

,id,name,description,description_story,description_risks,rewards,reward_tiers,min_reward,max_reward,category,goal,deadline,location,state,staff_pick,video,launched_at,created_at,rewards_processed,description_processed,description_story_processed,description_risks_processed,has_video,rewards_0,rewards_1,rewards_2,rewards_3,rewards_4,rewards_5,rewards_6,rewards_7,rewards_8,rewards_9,rewards_10,rewards_11,rewards_12,rewards_13,rewards_14,rewards_15,rewards_16,rewards_17,rewards_18,rewards_19,rewards_20,rewards_21,rewards_22,rewards_23,rewards_24,rewards_25,rewards_26,rewards_27,rewards_28,rewards_29,rewards_30,rewards_31,rewards_32,rewards_33,rewards_34,rewards_35,rewards_36,rewards_37,rewards_38,rewards_39,rewards_40,rewards_41,rewards_42,rewards_43,rewards_44,rewards_45,rewards_46,rewards_47,rewards_48,rewards_49,rewards_50,rewards_51,rewards_52,rewards_53,rewards_54,rewards_55,description_0,description_1,description_2,description_3,description_4,description_5,description_6,description_7,description_8,description_9,description_10,description_11,description_12,description_13,description_14,description_15,description_16,description_17,description_18,description_19,description_20,description_21,description_22,description_23,description_24,description_25,description_26,description_27,description_28,description_29,description_30,description_31,description_32,description_33,description_34,description_35,description_36,description_37,description_38,description_39,description_40,description_41,description_42,description_43,description_44,description_45,description_46,description_47,description_48,description_49,description_50,description_51,description_52,description_53,description_54,description_55,description_56,description_57,description_58,description_59,description_60,description_61,description_62,description_63,description_64,description_65,description_66,description_67,description_68,description_69,description_70,description_71,description_72,description_73,description_74,description_75,description_76,description_77,description_78,description_79,description_80,description_81,description_82,description_83,description_84,description_85,description_86,description_87,description_88,description_89,description_90,description_91,description_92,description_93,description_94,description_95,description_96,description_97,description_98,description_99,description_story_0,description_story_1,description_story_2,description_story_3,description_story_4,description_story_5,description_story_6,description_story_7,description_story_8,description_story_9,description_story_10,description_story_11,description_story_12,description_story_13,description_story_14,description_story_15,description_story_16,description_story_17,description_story_18,description_story_19,description_story_20,description_story_21,description_story_22,description_story_23,description_story_24,description_story_25,description_story_26,description_story_27,description_story_28,description_story_29,description_story_30,description_story_31,description_story_32,description_story_33,description_story_34,description_story_35,description_story_36,description_story_37,description_story_38,description_story_39,description_story_40,description_story_41,description_story_42,description_story_43,description_story_44,description_story_45,description_story_46,description_story_47,description_story_48,description_story_49,description_story_50,description_story_51,description_story_52,description_story_53,description_story_54,description_story_55,description_story_56,description_story_57,description_story_58,description_story_59,description_story_60,description_story_61,description_story_62,description_story_63,description_story_64,description_story_65,description_story_66,description_story_67,description_story_68,description_story_69,description_story_70,description_story_71,description_story_72,description_story_73,description_story_74,description_story_75,description_story_76,description_s

In [30]:
fuck = pd.read_csv('../kickstarter_train_final_20221021-185107.csv')

In [31]:
fuck.head()

,reward_tiers,min_reward,max_reward,goal,state,staff_pick,has_video,rewards_0,rewards_1,rewards_2,rewards_3,rewards_4,rewards_5,rewards_6,rewards_7,rewards_8,rewards_9,rewards_10,rewards_11,rewards_12,rewards_13,rewards_14,rewards_15,rewards_16,rewards_17,rewards_18,rewards_19,rewards_20,rewards_21,rewards_22,rewards_23,rewards_24,rewards_25,rewards_26,rewards_27,rewards_28,rewards_29,rewards_30,rewards_31,rewards_32,rewards_33,rewards_34,rewards_35,rewards_36,rewards_37,rewards_38,rewards_39,rewards_40,rewards_41,rewards_42,rewards_43,rewards_44,rewards_45,rewards_46,rewards_47,rewards_48,rewards_49,rewards_50,rewards_51,rewards_52,rewards_53,rewards_54,rewards_55,rewards_56,rewards_57,rewards_58,rewards_59,rewards_60,rewards_61,rewards_62,rewards_63,rewards_64,rewards_65,rewards_66,rewards_67,rewards_68,rewards_69,rewards_70,rewards_71,rewards_72,rewards_73,rewards_74,rewards_75,rewards_76,rewards_77,rewards_78,rewards_79,rewards_80,rewards_81,rewards_82,rewards_83,rewards_84,rewards_85,rewards_86,rewards_87,rewards_88,rewards_89,rewards_90,rewards_91,rewards_92,rewards_93,rewards_94,rewards_95,rewards_96,rewards_97,rewards_98,rewards_99,description_0,description_1,description_2,description_3,description_4,description_5,description_6,description_7,description_8,description_9,description_10,description_11,description_12,description_13,description_14,description_15,description_16,description_17,description_18,description_19,description_20,description_21,description_22,description_23,description_24,description_25,description_26,description_27,description_28,description_29,description_30,description_31,description_32,description_33,description_34,description_35,description_36,description_37,description_38,description_39,description_40,description_41,description_42,description_43,description_44,description_45,description_46,description_47,description_48,description_49,description_50,description_51,description_52,description_53,description_54,description_55,description_56,description_57,description_58,description_59,description_60,description_61,description_62,description_63,description_64,description_65,description_66,description_67,description_68,description_69,description_70,description_71,description_72,description_73,description_74,description_75,description_76,description_77,description_78,description_79,description_80,description_81,description_82,description_83,description_84,description_85,description_86,description_87,description_88,description_89,description_90,description_91,description_92,description_93,description_94,description_95,description_96,description_97,description_98,description_99,description_story_0,description_story_1,description_story_2,description_story_3,description_story_4,description_story_5,description_story_6,description_story_7,description_story_8,description_story_9,description_story_10,description_story_11,description_story_12,description_story_13,description_story_14,description_story_15,description_story_16,description_story_17,description_story_18,description_story_19,description_story_20,description_story_21,description_story_22,description_story_23,description_story_24,description_story_25,description_story_26,description_story_27,description_story_28,description_story_29,description_story_30,description_story_31,description_story_32,description_story_33,description_story_34,description_story_35,description_story_36,description_story_37,description_story_38,description_story_39,description_story_40,description_story_41,description_story_42,description_story_43,description_story_44,description_story_45,description_story_46,description_story_47,description_story_48,description_story_49,description_story_50,description_story_51,description_story_52,description_story_53,description_story_54,description_story_55,description_story_56,description_story_57,description_story_58,description_story_59,description_story_60,description_story_61,description_story_62,description_story_63,description_story_